In [2]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import os
import sys

sys.path.insert(0, os.getenv("HOME") + "/pycode/MscThesis/")
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)

import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
)
from amftrack.pipeline.paths.directory import (
    run_parallel_transfer,
    find_state,
    directory_scratch,
    directory_project,
    directory_archive,
)
import dropbox
from amftrack.transfer.functions.transfer import upload, zip_file
from subprocess import call
from tqdm.autonotebook import tqdm

/home/ipausers/bisot/pycode/MscThesis/amftrack/transfer/functions/transfer.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [67]:
path_code = os.getenv("HOME") + "/pycode/MscThesis/"
path_code

'/home/ipausers/bisot/pycode/MscThesis/'

In [3]:
directory = r"/run/user/357100554/gvfs/smb-share:server=sun.amolf.nl,share=shimizu-data,user=bisot/home-folder/oyartegalvez/Drive_AMFtopology/Healing/"
directory = r"/run/user/357100554/gvfs/smb-share:server=prince.amolf.nl,share=d$/Data/Prince/Images/"
# directory = r'/run/user/357100554/gvfs/smb-share:server=sun.amolf.nl,share=shimizu-data,user=bisot/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE_sync/'
# directory = r'/mnt/sun/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE_sync/'

# update_plate_info(directory)
all_folders = get_current_folders(directory)

In [29]:
directory2 = r"/mnt/sun/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE_sync_test"

In [51]:
origin = all_folders["total_path"].iloc[0]
target = directory2
origin, target

('/run/user/357100554/gvfs/smb-share:server=prince.amolf.nl,share=d$/Data/Prince/Images/20220307_0922_Plate01',
 '/mnt/sun/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE_sync_test')

In [59]:
def sync_fold(origin, target):
    cmd = f"rsync --update -avh {origin} {target}"
    # print(cmd)
    call(cmd, shell=True)

In [38]:
folders = all_folders

In [65]:
run_info = folders.copy()
target = directory2
folder_list = list(run_info["total_path"])
with tqdm(total=len(folder_list), desc="transferred") as pbar:
    for folder in folder_list:
        origin = folder
        sync_fold(origin, target)
        pbar.update(1)

transferred:   0%|          | 0/31 [00:00<?, ?it/s]

sending incremental file list
20220307_0922_Plate01/
20220307_0922_Plate01/Img/

sent 2.65K bytes  received 414 bytes  6.14K bytes/sec
total size is 3.15G  speedup is 1,025,353.35
sending incremental file list
20220308_0657_Plate01/
20220308_0657_Plate01/Img/

sent 139 bytes  received 27 bytes  332.00 bytes/sec
total size is 420  speedup is 2.53
sending incremental file list
20220308_0710_Plate01/
20220308_0710_Plate01/Img/
20220308_0710_Plate01/Img/Img_r05_c05.tif
20220308_0710_Plate01/Img/Img_r05_c06.tif
20220308_0710_Plate01/Img/Img_r05_c07.tif
20220308_0710_Plate01/Img/Img_r05_c08.tif
20220308_0710_Plate01/Img/Img_r05_c09.tif
20220308_0710_Plate01/Img/Img_r05_c10.tif
20220308_0710_Plate01/Img/Img_r05_c11.tif
20220308_0710_Plate01/Img/Img_r05_c12.tif
20220308_0710_Plate01/Img/Img_r05_c13.tif
20220308_0710_Plate01/Img/Img_r05_c14.tif
20220308_0710_Plate01/Img/Img_r06_c01.tif
20220308_0710_Plate01/Img/Img_r06_c02.tif
20220308_0710_Plate01/Img/Img_r06_c03.tif
20220308_0710_Plate01/Img/

In [43]:
!pip install checksumdir

In [46]:
checksumdir.dirhash(origin)

'c1d3fe323eeef81cb007785687cf62fd'

In [61]:
run_info = folders.copy()
target = directory2
folder_list = list(run_info["total_path"])
with tqdm(total=len(folder_list), desc="transferred") as pbar:
    for folder in folder_list:
        origin = folder
        copy = target + "/" + folder.split("/")[-1]
        break

transferred:   0%|          | 0/31 [00:00<?, ?it/s]

In [63]:
origin, copy

('/run/user/357100554/gvfs/smb-share:server=prince.amolf.nl,share=d$/Data/Prince/Images/20220307_0922_Plate01',
 '/mnt/sun/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE_sync_test/20220307_0922_Plate01')

In [64]:
import checksumdir

checksumdir.dirhash(origin), checksumdir.dirhash(copy)

('14d575547c25462ce0b349a91b0b8dc9', '14d575547c25462ce0b349a91b0b8dc9')

In [56]:
checksumdir.dirhash(origin)

'14d575547c25462ce0b349a91b0b8dc9'

In [57]:
checksumdir.dirhash(target)

'74fd3bba045fb73e7d500d2be0bc86fe'

In [ ]:
def get_current_folders(directory, file_metadata=False):
    if directory == "dropbox":
        data = []
        dbx = dropbox.Dropbox(API)
        response = dbx.files_list_folder("", recursive=True)
        # for fil in response.entries:
        listfiles = []
        listjson = []
        while response.has_more:
            listfiles += [
                file for file in response.entries if file.name.split(".")[-1] == "zip"
            ]
            listjson += [
                file.path_lower
                for file in response.entries
                if file.name.split(".")[-1] == "json"
            ]

            response = dbx.files_list_folder_continue(response.cursor)
        listfiles += [
            file for file in response.entries if file.name.split(".")[-1] == "zip"
        ]
        listjson += [
            file.path_lower
            for file in response.entries
            if file.name.split(".")[-1] == "json"
        ]
        # print([((file.path_lower.split(".")[0]) + "_info.json") for file in listfiles if (file.name.split(".")[-1] == "zip") &
        #        (((file.path_lower.split(".")[0]) + "_info.json") not in listjson)])
        listfiles.reverse()
        if file_metadata:
            return listfiles

In [ ]:
files = get_current_folders("dropbox", True)
files[0]